# Module 9 Case Study Problem 1 Recommendation engine using SVD in python

In [1]:
import pandas as pd
import numpy as np

In [2]:
movieratings = pd.read_csv(r"H:\Intellipaat data science\Python\Python for Data Science\Module 9\Case Study Problem 1 Movie Recommendation engine\ratings.csv")

In [3]:
movieratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
movies = pd.read_csv(r"H:\Intellipaat data science\Python\Python for Data Science\Module 9\Case Study Problem 1 Movie Recommendation engine\movies.csv")

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
n_users = movieratings.userId.unique().shape[0]

n_movies = movieratings.movieId.unique().shape[0]

print(f'Number of users = {n_users} and Number of movies = {n_movies}')

Number of users = 7120 and Number of movies = 14026


In [7]:
#To create a rating matrix for the 'ratings' dataset

In [8]:
Ratings = movieratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [11]:
# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(movieratings[['userId', 'movieId', 'rating']], reader)

# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8458  0.8472  0.8428  0.8453  0.0018  
MAE (testset)     0.6478  0.6496  0.6451  0.6475  0.0018  
Fit time          42.45   44.96   48.71   45.37   2.57    
Test time         4.64    3.72    3.53    3.96    0.48    


{'test_rmse': array([0.84578743, 0.84722482, 0.84278717]),
 'test_mae': array([0.64775704, 0.64955619, 0.64509932]),
 'fit_time': (42.449485063552856, 44.960174560546875, 48.70666313171387),
 'test_time': (4.636250734329224, 3.7154712677001953, 3.531437635421753)}

In [13]:
movieratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [14]:
# To find all the movies rated as more than 4 stars by user with userId = 1

In [17]:
ratings_1 = movieratings[(movieratings['userId'] == 5) & (movieratings['rating'] == 5)]
ratings_1 = ratings_1.set_index('movieId')
ratings_1 = ratings_1.join(movies)['title']
ratings_1.head(10)

movieId
11                    Dracula: Dead and Loving It (1995)
62     Don't Be a Menace to South Central While Drink...
141                                         Gospa (1995)
150                                Addiction, The (1995)
260                             Ladybird Ladybird (1994)
318    Strawberry and Chocolate (Fresa y chocolate) (...
364                                      Maverick (1994)
368                                 Reality Bites (1994)
377                      When a Man Loves a Woman (1994)
380                                   Bad Company (1995)
Name: title, dtype: object

In [18]:
#Train an SVD to predict ratings for user with userId = 1

In [19]:
# Create a shallow copy for the movies dataset
user_5 = movies.copy()

#Reset the index for user_5 dataset
user_5 = user_5.reset_index()



# getting full dataset
data = Dataset.load_from_df(movieratings[['userId', 'movieId', 'rating']], reader)


#create a training set for svd
trainset = data.build_full_trainset()
svd.fit(trainset)

#Predict the ratings for user1
user_5['Estimate_Score'] = user_5['movieId'].apply(lambda x: svd.predict(1, x).est)

#Drop extra columns from the user1 data frame
user_5 = user_5.drop(['movieId','genres','index'], axis = 1)

# Sort predicted ratings for user1 in descending order
user_5 = user_5.sort_values('Estimate_Score', ascending=False)

#Print top 10 recommendations
print(user_5.head(10))

                                                   title  Estimate_Score
4897   Lord of the Rings: The Fellowship of the Ring,...        4.639742
5853       Lord of the Rings: The Two Towers, The (2002)        4.634966
7041   Lord of the Rings: The Return of the King, The...        4.632128
6172           Day for Night (La Nuit Américaine) (1973)        4.554955
1049                             Murder, My Sweet (1944)        4.516959
15208                                      Cosmos (1980)        4.505420
6501                                   Umberto D. (1952)        4.473309
2773                            Stop Making Sense (1984)        4.472626
7356                             Band of Brothers (2001)        4.446868
108                                    Braveheart (1995)        4.445370


# END OF ASSIGNMENT